In [134]:
import json
import pandas as pd
import os
import re
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', -1)

In [2]:
all_recipes = []
for file in os.listdir("./"):
    if ".json" in file:
        with open(file, "r") as scraped:
            loaded = json.load(scraped)
            for recipe in loaded:
                all_recipes.append(recipe)

In [6]:
with open("recipes.json", "w") as all_jsons:
    json.dump(all_recipes, all_jsons)

with open("recipes.json", "r") as all_jsons:
    loaded = json.load(all_jsons)
loaded[:2]

In [8]:
len(loaded)

21772

In [9]:
r = pd.DataFrame(all_recipes)

In [10]:
r["likes"][r["likes"] == ""] = "0"
r["raters"][r["raters"] == ""] = "0"

c:\users\helen\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\helen\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
r.likes = r.likes.astype(float)
r.raters = r.raters.astype(float)
print(r.dtypes)

categories      object 
difficulty      object 
duration        object 
ingredients     object 
instructions    object 
likes           float64
link            object 
name            object 
raters          float64
rating          float64
tags            object 
dtype: object


In [22]:
r["rating2"] = (r["raters"] * r["rating"] + (r["likes"] / 2) * 5) / (r["raters"] + (r["likes"]/ 2))
max_raters = (r["likes"] + (r["raters"] / 2)).max()
max_raters

12768.0

In [23]:
mean_rating = r["rating2"].mean()
mean_rating

4.895507005530549

In [17]:
r["rating3"] = ((r["likes"] + (r["raters"] / 2)) / max_raters) * mean_rating + (1 - ((r["likes"] + (r["raters"] / 2)) / max_raters)) * r["rating2"]
r["rating3"].max()

In [19]:
r.sort_values("rating3", ascending = False, inplace = True)

In [41]:
results_name = r[r["name"].str.lower().str.contains("pizza")] # Name in singular
#winner_indices.append(results_name[:1].index)
results_name[:1]

,categories,difficulty,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3
6318,"[Dania i przekąski, Zupy]",Łatwe,30 minut,"{'kiełbasa pepperoni': '200 g', 'rosół wołowy': '1 szt.', 'cebula': '2 szt.', 'ząbki czosnku': '4 szt.', 'pomidory z puszki': '800 g', 'marchew': '2 szt.', 'bulion': '2 szklanki', 'ser mozzarella': '150 g', 'oliwa': '40 ml', 'oregano z turcji': '1 łyżeczka'}","Krok 1\nKiełbasę pokrój w cienkie plasterki.\nKrok 2\nCebulkę i czosnek posiekaj, podsmaż na rozgrzanym tłuszczu\nKrok 3\nDodaj kawałki pomidorów wraz z zalewą i starte na tarce marchewki.\nKrok 4\nRozgnieć sos drewnianą łyżką, podlej wodą, 500ml. Dodaj kostkę Knorr.\nKrok 5\nNastępnie wsyp pepperoni i duś na małym ogniu przez ok.15-20min.\nKrok 6\nRozlej do misek i do każdej dodaj startą mozzarellę. Zapiecz w nagrzanym do 220 stopni piekarniku przez kilka minut, do czasu aż ser się rozpłynie.\nKrok 7\nUdekoruj zieleniną i smaczną zupę a la pizza podawaj z chlebem.\n\nOregano z Turcji\nKlimat Turcji sprzyja uprawie roślin, w tym ziół, które charakteryzują się intensywnym aromatem. Oregano – bliski brat majeranku – jest tego przykładem. Dania tureckie są dziś obecne w wielu kuchniach krajów europejskich, tak jak np. pilaw, czyli potrawa z ryżu z dodatkiem innych składników.",0.0,https://www.przepisy.pl/przepis/smaczna-zupa-a-la-pizza-3,Smaczna zupa a la pizza,1.0,5.0,"[ciasto na pizzę, zupy, mięso, cebula, kuchnia włoska, pieczenie, szybkie zupy, frisco, pomysł na zupę]",5.0,4.999996


In [69]:
r_index = list(r.index.values)
r_categories = list(r.categories.values)
r_categories_lower = []
for recipe in r_categories:
    categories_lower = []
    for category in recipe:
        category = category.lower()
        categories_lower.append(category)
    r_categories_lower.append(categories_lower)
r_categories_tuple = list(zip(r_index, r_categories_lower))

[(16364, ['dania i przekąski', 'sosy']), (389, ['dania i przekąski', 'zupy'])]

In [97]:
results_categories_tuple = [recipe[0] for recipe in r_categories_tuple if "pizze" in recipe[1]] # Category in plural or uncount
results_categories_after_tuple = pd.DataFrame(results_categories_tuple).set_index(0)
results_categories = pd.merge(results_categories_after_tuple, r, left_index=True, right_index=True)
results_categories.head()

[2742, 2724]

In [101]:
r_tags = list(r.tags.values)
r_tags_tuple = list(zip(r_index, r_tags))
r_tags_tuple[:2]

[(16364,
  ['ryby',
   'sosy',
   'sosy',
   'kuchnia grecka',
   'prosty przepis',
   'przepisy z 5 składników',
   'szybkie danie']),
 (389,
  ['zupy',
   'cebula',
   'sałata lodowa',
   'ogórek',
   'dania do pracy',
   'frisco',
   'pomysł na zupę',
   'szybkie danie'])]

In [107]:
results_tags_tuple = [recipe[0] for recipe in r_tags_tuple if "pizze" in recipe[1]] # Tag in plural or uncountable
results_tags_after_tuple = pd.DataFrame(results_tags_tuple).set_index(0)
results_tags = pd.merge(results_tags_after_tuple, r, left_index=True, right_index=True)
results_tags.head()

,categories,difficulty,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3
2742,"[Dania i przekąski, Pizze]",Łatwe,45 minut,"{'pizza (ciasto + sos)': '1 opak.', 'krewetki black tiger': '10 szt.', 'szynka prosciutto': '10 plastrów', 'pesto': '1 szt.', 'mozarella tarta': '100 g', 'przyprawa w mini kostkach czosnek': '1 szt.'}",Krok 1\nCiasto i sos przygotowujemy zgodnie z zaleceniami na opakowaniu.\nKrok 2\nKiedy ciasto wyrośnie rozciągamy je. Na cieście rozsmarowujemy sos a następnie posypujemy go mozarellą.\nKrok 3\nKażdą krewetkę owijamy połową plasterka szynki i układamy na pizzy. Między krewetkami dajemy po 1/2 łyżeczki pesto. Całość posypujemy posiekaną papryczką i rozkruszoną kosteczką czosnkową.\nKrok 4\nPizzę wstawiamy do piekarnika nagrzanego do 220 stopni i pieczmy ok 15-20 min.,0.0,https://www.przepisy.pl/przepis/pizza-z-krewetkami-pesto-i-prosciutto,"Pizza z krewetkami, pesto i prosciutto",1.0,5.0,"[ciasto na pizzę, mięso, owoce morza, prosty przepis, pieczenie, pizze]",5.0,4.999996
2724,"[Dania i przekąski, Pizze]",Średnie,45 minut,"{'mozzarella': '300 g', 'bazylia': '0,2 szklanki', 'sos pomidorowy': '5 łyżek', 'ciasto:': '', 'mąka': '1 kg', 'olej': '50 ml', 'drożdże': '8 g', 'sos:': '', 'koncentrat pomidorowy': '100 g', 'cukier': '1 łyżeczka', 'sól': '1 łyżeczka', 'oregano': '2 łyżeczki', 'woda': '30 ml', 'przyprawa w mini kostkach czosnek': '2 szt.'}","Krok 1\nMozzarellę pokrój w cienkie plastry.\nKrok 2\nSkładniki na ciasto do pizzy połącz ze sobą dobrze wyrabiając na stolnicy, odstaw na 10 min do lodówki skropione wodą, po 10 min wyjmij do wyrośnięcia w ciepłym miejscu na 20 min.\nKrok 3\nRozgrzej piekarnik do 180 stopni. Następnie podziel ciasto na pół.\nKrok 4\nNa oprószonym mąką blacie uformuj z ciasta 30 centymetrowe placki i ułóż na posmarowanych oliwą blachach. Polej każdy sosem, posyp startym serem i ułóż plastry mozzarelli. Posyp mini kostkami Knorr.\nKrok 5\nPiecz około 15 minut, aż skórka zarumieni się na złotu. Przed podaniem obficie posyp bazylią.",1.0,https://www.przepisy.pl/przepis/pizza-napoletana,Pizza napolitana,0.0,0.0,"[ciasto na pizzę, kuchnia włoska, mozzarella, pieczenie, pizze, danie z serem]",5.0,4.999992
2725,"[Dania i przekąski, Pizze]",Średnie,45 minut,"{'ząbek czosnku': '1 szt.', 'oliwa z oliwek': '2 łyżki', 'pokrojona w paseczki bazylia (kilka listków)': '4 łyżki', 'parmezan starty': '5 łyżek', 'mozzarella': '250 g', 'ciasto na pizze': '500 g', 'ricotta': '150 g', 'ser pecorino': '150 g'}","Krok 1\nCzosnek zmiażdż, wymieszaj z oliwą i posmaruj gotowe ciasto na pizze.\nKrok 2\nMozzarellę pokrój w paski i ułóż na cieście. Posyp serem pecorino.\nKrok 3\nNałóż ricottę i posyp startym parmezanem.\nKrok 4\nPiecz około 20 minut w temperaturze 220 stopni.\nKrok 5\nW czasie pieczenia smaruj pizzę oliwą z czosnkiem. Posyp posiekaną bazylią.",1.0,https://www.przepisy.pl/przepis/pizza-serowa-z-bazylia,Pizza serowa z bazylią,0.0,0.0,"[ciasto na pizzę, kuchnia włoska, mozzarella, kuchnia śródziemnomorska, zapiekanki, pizze, ricotta, danie z serem]",5.0,4.999992
2723,"[Dania i przekąski, Pizze]",Łatwe,90 minut,"{'pomidor': '300 g', 'pieprz': '1 szczypta', 'sól': '1 szczypta', 'cukier': '2 łyżki', 'mleko': '4 łyżki', 'czarne oliwki': '1 opak.', 'mąka': '125 g', 'oliwa': '1 łyżka', 'bazylia': '1 łyżeczka', 'mozzarella': '150 g', 'drożdże': '10 g', 'salami': '75 g', 'pasta sardelowa': '3 łyżeczki'}","Krok 1\nDrożdże pokrusz, wymieszaj z cukrem i ciepłym mlekiem. Dodaj szczyptę soli i odrobinę oliwy, zagnieć na jednolite, gładkie cisto i zostaw w ciepłym miejscu do wyrośnięcia.\nKrok 2\nPonownie zagnieć, rozwałkuj i przełóż na wysmarowanej tłuszczem blaszce.\nKrok 3\nosmaruj ciasto warstwą pasty sardelowej. Pokrój pomidory w plastry i ułóż na cieście. Oprósz solą, pieprzem i ziołami.\nKrok 4\nPrzykryj warstwą salami i oliwkami, skrop oliwą i piecz pizzę przez 30-40 minut w temperaturze 220 stopni.",1.0,https://www.przepisy.pl/przepis/pizza-salami,Pizza salami,0.0,0.0,"

In [109]:
r_ingredients = list(r.ingredients.values)
r_ingredients_tuple = list(zip(r_index, r_ingredients))
r_ingredients_tuple[:2]

[(16364,
  {'masło': '100 g',
   'zielona pietruszka': '1 łyżka',
   'kapary słoiczek': '1 opak.',
   'pasta anchois': '50 g',
   'pieprz czarny z wietnamu mielony': ''}),
 (389,
  {'cebula': '50 g',
   'czosnek - ząbek': '1 szt.',
   'oliwa': '20 g',
   'ogórek': '250 g',
   'sałata lodowa': '50 g',
   'woda': '700 g',
   'ryż': '50 g',
   'pieprz': '1 łyżeczka',
   'koperek': '1 łyżka',
   'posiekana natka pietruszki': '1 łyżka',
   'bulion cielęcy': '1 szt.',
   'delikat przyprawa uniwersalna': '1,5 łyżeczki'})]

In [111]:
results_ingredients_tuple = [recipe[0] for recipe in r_ingredients_tuple if "pizza" in recipe[1]]
results_ingredients_after_tuple = pd.DataFrame(results_ingredients_tuple).set_index(0)
results_ingredients = pd.merge(results_ingredients_after_tuple, r, left_index=True, right_index=True)
results_ingredients.head()

,categories,difficulty,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3
6098,"[Dania i przekąski, Dania główne]",Łatwe,45 minut,"{'pizza': '1 opak.', 'papryka czerwona': '1 szt.', 'cebula': '1 szt.', 'olej słonecznikowy': '2 łyżki', 'biała kiełbasa': '3 szt.'}","Krok 1\nCiasto i sos na pizzę przygotowałam zgodnie z instrukcją na opakowaniu.\nKrok 2\nKiełbasę, paprykę i cebulę pokroiłam drobno i podsmażyłam na oleju.\nKrok 3\nPodsmażoną kiełbasę z warzywami dusiłam z sosem do pizzy do mocnego odparowania.\nKrok 4\nCiasto na pizzę podzieliłam na 4 części, na każdej z nich ułożyłam 1/4 nadzienia i ulepiłam z nich duże pierogi dociskając brzegi widelcem.\nKrok 5\nPiekłam 20 minut w piekarniku nagrzanym do 200 stopni.",11.0,https://www.przepisy.pl/przepis/calzone-2,Calzone,0.0,0.0,"[warzywa, mięso, dania główne, cebula, kuchnia włoska, zapiekanki, pieczenie, przepisy z 5 składników]",5.0,4.999910
2744,"[Dania i przekąski, Pizze]",Łatwe,60 minut,"{'pizza': '1 szt.', 'cebula': '1 szt.', 'chorizo': '150 g', 'tarty ser żółty': '2 szklanki', 'sos spaghetti toscana': '1 szt.'}",Krok 1\nSpód do pizzy przygotowujemy zgodnie z instrukcją producenta.\nKrok 2\nNa gotowym spodzie równomiernie rozprowadzamy pół słoika sosu pomidorowego.\nKrok 3\nNa nim układamy pokrojone w plastry cebulę i pomidora.\nKrok 4\nNa nie układamy pokrojoną w plasterki kiełbasę chorizo.\nKrok 5\nCałość posypujemy serem i skrapiamy oliwą.\nKrok 6\nPieczemy około 20-30 minut w 180 st. C. Smacznego!,4.0,https://www.przepisy.pl/przepis/pizza-z-chorizo,Pizza z chorizo,2.0,3.0,"[ciasto na pizzę, mięso, cebula, kuchnia włoska, pizze, przepisy z 5 składników]",4.0,4.000351


In [112]:
search_results = pd.concat([results_name, results_categories, results_tags, results_ingredients])
search_results.head()

,categories,difficulty,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3
6318,"[Dania i przekąski, Zupy]",Łatwe,30 minut,"{'kiełbasa pepperoni': '200 g', 'rosół wołowy': '1 szt.', 'cebula': '2 szt.', 'ząbki czosnku': '4 szt.', 'pomidory z puszki': '800 g', 'marchew': '2 szt.', 'bulion': '2 szklanki', 'ser mozzarella': '150 g', 'oliwa': '40 ml', 'oregano z turcji': '1 łyżeczka'}","Krok 1\nKiełbasę pokrój w cienkie plasterki.\nKrok 2\nCebulkę i czosnek posiekaj, podsmaż na rozgrzanym tłuszczu\nKrok 3\nDodaj kawałki pomidorów wraz z zalewą i starte na tarce marchewki.\nKrok 4\nRozgnieć sos drewnianą łyżką, podlej wodą, 500ml. Dodaj kostkę Knorr.\nKrok 5\nNastępnie wsyp pepperoni i duś na małym ogniu przez ok.15-20min.\nKrok 6\nRozlej do misek i do każdej dodaj startą mozzarellę. Zapiecz w nagrzanym do 220 stopni piekarniku przez kilka minut, do czasu aż ser się rozpłynie.\nKrok 7\nUdekoruj zieleniną i smaczną zupę a la pizza podawaj z chlebem.\n\nOregano z Turcji\nKlimat Turcji sprzyja uprawie roślin, w tym ziół, które charakteryzują się intensywnym aromatem. Oregano – bliski brat majeranku – jest tego przykładem. Dania tureckie są dziś obecne w wielu kuchniach krajów europejskich, tak jak np. pilaw, czyli potrawa z ryżu z dodatkiem innych składników.",0.0,https://www.przepisy.pl/przepis/smaczna-zupa-a-la-pizza-3,Smaczna zupa a la pizza,1.0,5.0,"[ciasto na pizzę, zupy, mięso, cebula, kuchnia włoska, pieczenie, szybkie zupy, frisco, pomysł na zupę]",5.0,4.999996
2189,"[Dania i przekąski, Kanapki]",Łatwe,15 minut,"{'duże bułki': '2 szt.', 'żółty ser': '4 plastry', 'salami': '8 plastrów', 'sos barbecue': '', 'masło': '1 łyżka'}","Krok 1\nNagrzej piekarnik.\nKrok 2\nPrzekrój bułki na pół, posmaruj masłem, połóż salami.\nKrok 3\nWłóż do piekarnika na 15 minut. Na chrupiące bułeczki połóż ser i polej je sosem barbecue.",0.0,https://www.przepisy.pl/przepis/kanapki-a-la-pizza,Kanapki a’la pizza,1.0,5.0,"[ciasto na pizzę, kuchnia amerykańska, prosty przepis, pieczenie, kanapki, przepisy z 5 składników, szybkie danie]",5.0,4.999996
2742,"[Dania i przekąski, Pizze]",Łatwe,45 minut,"{'pizza (ciasto + sos)': '1 opak.', 'krewetki black tiger': '10 szt.', 'szynka prosciutto': '10 plastrów', 'pesto': '1 szt.', 'mozarella tarta': '100 g', 'przyprawa w mini kostkach czosnek': '1 szt.'}",Krok 1\nCiasto i sos przygotowujemy zgodnie z zaleceniami na opakowaniu.\nKrok 2\nKiedy ciasto wyrośnie rozciągamy je. Na cieście rozsmarowujemy sos a następnie posypujemy go mozarellą.\nKrok 3\nKażdą krewetkę owijamy połową plasterka szynki i układamy na pizzy. Między krewetkami dajemy po 1/2 łyżeczki pesto. Całość posypujemy posiekaną papryczką i rozkruszoną kosteczką czosnkową.\nKrok 4\nPizzę wstawiamy do piekarnika nagrzanego do 220 stopni i pieczmy ok 15-20 min.,0.0,https://www.przepisy.pl/przepis/pizza-z-krewetkami-pesto-i-prosciutto,"Pizza z krewetkami, pesto i prosciutto",1.0,5.0,"[ciasto na pizzę, mięso, owoce morza, prosty przepis, pieczenie, pizze]",5.0,4.999996
4967,"[Dania i przekąski, Przystawki]",Średnie,60 minut,"{'mąka': '1 kg', 'drożdże': '10 dag', 'margaryna': '1 opak.', 'żółtka': '2 szt.', 'sól': '1 łyżeczka', 'oregano': '1 szczypta', 'cukier': '1 łyżeczka', 'mleko': '0,5 l'}","Krok 1\nPodane składniki mieszamy ze sobą formując ciasto.\nKrok 2\nFarsz: 1 pierś z kurczaka, pieczarki, por, papryka, cebula 2x, 25 dkg żółtego sera, kocentrat pomidorowy (lub inne wg uznania)\nKrok 3\nZ ciasta formujemy pierożki, nadziewamy je wybranym przez siebie farszem i pieczemy w piekarniku przez około 20 minut ( do zaróżowienia ciasta)",1.0,https://www.przepisy.pl/przepis/pierozek-pizza,Pierożek pizza,0.0,0.0,"[ciasto na pizzę, drób, przystawki, prosty przepis, pieczenie, frisco]",5.0,4.999992
2724,"[Dania i przekąski, Pizze]",Średnie,45 minut,"{'mozzarella': '300 g', 'bazylia': '0,2 szklanki', 'sos pomidorowy': '5 łyżek', 'ciasto:': '', 'mąka': '1 kg', 'olej': '50 ml', 'drożdże': '8 g', 'sos:': '', 'koncentrat pomidorowy': '10

In [114]:
search_results["ingr_len"] = search_results["ingredients"].apply(len)
search_results.head()

,categories,difficulty,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3,ingr_len
6318,"[Dania i przekąski, Zupy]",Łatwe,30 minut,"{'kiełbasa pepperoni': '200 g', 'rosół wołowy': '1 szt.', 'cebula': '2 szt.', 'ząbki czosnku': '4 szt.', 'pomidory z puszki': '800 g', 'marchew': '2 szt.', 'bulion': '2 szklanki', 'ser mozzarella': '150 g', 'oliwa': '40 ml', 'oregano z turcji': '1 łyżeczka'}","Krok 1\nKiełbasę pokrój w cienkie plasterki.\nKrok 2\nCebulkę i czosnek posiekaj, podsmaż na rozgrzanym tłuszczu\nKrok 3\nDodaj kawałki pomidorów wraz z zalewą i starte na tarce marchewki.\nKrok 4\nRozgnieć sos drewnianą łyżką, podlej wodą, 500ml. Dodaj kostkę Knorr.\nKrok 5\nNastępnie wsyp pepperoni i duś na małym ogniu przez ok.15-20min.\nKrok 6\nRozlej do misek i do każdej dodaj startą mozzarellę. Zapiecz w nagrzanym do 220 stopni piekarniku przez kilka minut, do czasu aż ser się rozpłynie.\nKrok 7\nUdekoruj zieleniną i smaczną zupę a la pizza podawaj z chlebem.\n\nOregano z Turcji\nKlimat Turcji sprzyja uprawie roślin, w tym ziół, które charakteryzują się intensywnym aromatem. Oregano – bliski brat majeranku – jest tego przykładem. Dania tureckie są dziś obecne w wielu kuchniach krajów europejskich, tak jak np. pilaw, czyli potrawa z ryżu z dodatkiem innych składników.",0.0,https://www.przepisy.pl/przepis/smaczna-zupa-a-la-pizza-3,Smaczna zupa a la pizza,1.0,5.0,"[ciasto na pizzę, zupy, mięso, cebula, kuchnia włoska, pieczenie, szybkie zupy, frisco, pomysł na zupę]",5.0,4.999996,10
2189,"[Dania i przekąski, Kanapki]",Łatwe,15 minut,"{'duże bułki': '2 szt.', 'żółty ser': '4 plastry', 'salami': '8 plastrów', 'sos barbecue': '', 'masło': '1 łyżka'}","Krok 1\nNagrzej piekarnik.\nKrok 2\nPrzekrój bułki na pół, posmaruj masłem, połóż salami.\nKrok 3\nWłóż do piekarnika na 15 minut. Na chrupiące bułeczki połóż ser i polej je sosem barbecue.",0.0,https://www.przepisy.pl/przepis/kanapki-a-la-pizza,Kanapki a’la pizza,1.0,5.0,"[ciasto na pizzę, kuchnia amerykańska, prosty przepis, pieczenie, kanapki, przepisy z 5 składników, szybkie danie]",5.0,4.999996,5
2742,"[Dania i przekąski, Pizze]",Łatwe,45 minut,"{'pizza (ciasto + sos)': '1 opak.', 'krewetki black tiger': '10 szt.', 'szynka prosciutto': '10 plastrów', 'pesto': '1 szt.', 'mozarella tarta': '100 g', 'przyprawa w mini kostkach czosnek': '1 szt.'}",Krok 1\nCiasto i sos przygotowujemy zgodnie z zaleceniami na opakowaniu.\nKrok 2\nKiedy ciasto wyrośnie rozciągamy je. Na cieście rozsmarowujemy sos a następnie posypujemy go mozarellą.\nKrok 3\nKażdą krewetkę owijamy połową plasterka szynki i układamy na pizzy. Między krewetkami dajemy po 1/2 łyżeczki pesto. Całość posypujemy posiekaną papryczką i rozkruszoną kosteczką czosnkową.\nKrok 4\nPizzę wstawiamy do piekarnika nagrzanego do 220 stopni i pieczmy ok 15-20 min.,0.0,https://www.przepisy.pl/przepis/pizza-z-krewetkami-pesto-i-prosciutto,"Pizza z krewetkami, pesto i prosciutto",1.0,5.0,"[ciasto na pizzę, mięso, owoce morza, prosty przepis, pieczenie, pizze]",5.0,4.999996,6
4967,"[Dania i przekąski, Przystawki]",Średnie,60 minut,"{'mąka': '1 kg', 'drożdże': '10 dag', 'margaryna': '1 opak.', 'żółtka': '2 szt.', 'sól': '1 łyżeczka', 'oregano': '1 szczypta', 'cukier': '1 łyżeczka', 'mleko': '0,5 l'}","Krok 1\nPodane składniki mieszamy ze sobą formując ciasto.\nKrok 2\nFarsz: 1 pierś z kurczaka, pieczarki, por, papryka, cebula 2x, 25 dkg żółtego sera, kocentrat pomidorowy (lub inne wg uznania)\nKrok 3\nZ ciasta formujemy pierożki, nadziewamy je wybranym przez siebie farszem i pieczemy w piekarniku przez około 20 minut ( do zaróżowienia ciasta)",1.0,https://www.przepisy.pl/przepis/pierozek-pizza,Pierożek pizza,0.0,0.0,"[ciasto na pizzę, drób, przystawki, prosty przepis, pieczenie, frisco]",5.0,4.999992,8
2724,"[Dania i przekąski, Pizze]",Średnie,45 minut,"{'mozzarella': '300 g', 'bazylia': '0,2 szklanki', 'sos pomidorowy': '5 łyżek', 'ciasto:': '', 'mąka': '1 kg', 'olej': '50 ml', 'drożdże': '8 g', 'sos:': '', 'koncentra

In [117]:
ingr_min = 2
if len(search_results[search_results["ingr_len"] == ingr_min]) == 0:
    ingr_min = 3
    while len(search_results[search_results["ingr_len"] == ingr_min]) == 0:
        ingr_min += 1

3


In [132]:
smallest_ingredients = search_results[search_results["ingr_len"] == ingr_min]
max_rating_ingredients = smallest_ingredients["rating3"].max()
best_ingredients = smallest_ingredients[smallest_ingredients["rating3"] == max_rating_ingredients]
best_ingredients

,categories,difficulty,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3,ingr_len
18066,"[WEGE, Dania wegetariańskie, Przekąski i dodatki wegetariańskie]",Łatwe,15 minut,"{'gotowe ciasto na pizzę': '0 g', 'parmezan (lub inny twardy ser)': '100 g', 'naturalnie smaczne spaghetti bolognese': '1 opak.'}","Krok 1\nW małym rondelku pomieszaj 200 ml wody z Knorr Naturalnie smaczne. Zagotuj.\nKrok 2\nCiasto na pizzę rozwałkuj na stolnicy oprószonej mąką. Pokrój na 2 cm paski dowolnej długości.\nKrok 3\nPaski posmaruj przygotowanym sosem pomidorowym, oprósz tartym serem.\nKrok 4\nGotowe paski poskręcaj w długie twisty, ułóż na blasze z piekarnika wyłożonej pergaminem.\nKrok 5\nWstaw do nagrzanego do 200° C piekarnika na około 15 minut, aż staną się złotawe i chrupiące.\n\nCiasto z pizzy można zastąpić ciastem francuskim.\nUpewnij się, że ser, którego używasz, zawiera podpuszczkę mikrobiologiczną, a nie naturalną.",1745.0,https://www.przepisy.pl/przepis/pizza-sticks,Pizza sticks - VIDEO,203.0,4.5,"[ciasto na pizzę, przekąski na imprezę, przystawki na zimno, dania z piekarnika, szybkie i tanie przekąski na imprezę, pieczenie, szybkie dania, przekąski, przekąski na karnawał, bez mięsa, dania wegetariańskie, przekąski wegetariańskie, dodatki wegetariańskie, przekąski na imprezę, przystawki na zimno, ulubione dania, przepisy z 3 składników, frisco, pomysł na danie bez mięsa, szybkie danie, danie bez mięsa, impreza]",4.905625,4.904162,3


In [129]:
r_difficulty = list(r.difficulty.values)
different_difficulty = []
for difficulty in r_difficulty:
    if difficulty not in different_difficulty:
        different_difficulty.append(difficulty)
print(different_difficulty)

['Łatwe', 'Średnie', 'Trudne']


In [130]:
difficulty = "Łatwe"
if len(search_results[search_results["difficulty"] == difficulty]) == 0:
    difficulty = "Średnie"
    if len(search_results[search_results["difficulty"] == difficulty]) == 0:
        difficulty = "Trudne"

Łatwe


In [131]:
easiest = search_results[search_results["difficulty"] == difficulty]
max_rating_easiest = easiest["rating3"].max()
best_easiest = easiest[easiest["rating3"] == max_rating_easiest]
best_easiest

,categories,difficulty,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3,ingr_len
6318,"[Dania i przekąski, Zupy]",Łatwe,30 minut,"{'kiełbasa pepperoni': '200 g', 'rosół wołowy': '1 szt.', 'cebula': '2 szt.', 'ząbki czosnku': '4 szt.', 'pomidory z puszki': '800 g', 'marchew': '2 szt.', 'bulion': '2 szklanki', 'ser mozzarella': '150 g', 'oliwa': '40 ml', 'oregano z turcji': '1 łyżeczka'}","Krok 1\nKiełbasę pokrój w cienkie plasterki.\nKrok 2\nCebulkę i czosnek posiekaj, podsmaż na rozgrzanym tłuszczu\nKrok 3\nDodaj kawałki pomidorów wraz z zalewą i starte na tarce marchewki.\nKrok 4\nRozgnieć sos drewnianą łyżką, podlej wodą, 500ml. Dodaj kostkę Knorr.\nKrok 5\nNastępnie wsyp pepperoni i duś na małym ogniu przez ok.15-20min.\nKrok 6\nRozlej do misek i do każdej dodaj startą mozzarellę. Zapiecz w nagrzanym do 220 stopni piekarniku przez kilka minut, do czasu aż ser się rozpłynie.\nKrok 7\nUdekoruj zieleniną i smaczną zupę a la pizza podawaj z chlebem.\n\nOregano z Turcji\nKlimat Turcji sprzyja uprawie roślin, w tym ziół, które charakteryzują się intensywnym aromatem. Oregano – bliski brat majeranku – jest tego przykładem. Dania tureckie są dziś obecne w wielu kuchniach krajów europejskich, tak jak np. pilaw, czyli potrawa z ryżu z dodatkiem innych składników.",0.0,https://www.przepisy.pl/przepis/smaczna-zupa-a-la-pizza-3,Smaczna zupa a la pizza,1.0,5.0,"[ciasto na pizzę, zupy, mięso, cebula, kuchnia włoska, pieczenie, szybkie zupy, frisco, pomysł na zupę]",5.0,4.999996,10
2189,"[Dania i przekąski, Kanapki]",Łatwe,15 minut,"{'duże bułki': '2 szt.', 'żółty ser': '4 plastry', 'salami': '8 plastrów', 'sos barbecue': '', 'masło': '1 łyżka'}","Krok 1\nNagrzej piekarnik.\nKrok 2\nPrzekrój bułki na pół, posmaruj masłem, połóż salami.\nKrok 3\nWłóż do piekarnika na 15 minut. Na chrupiące bułeczki połóż ser i polej je sosem barbecue.",0.0,https://www.przepisy.pl/przepis/kanapki-a-la-pizza,Kanapki a’la pizza,1.0,5.0,"[ciasto na pizzę, kuchnia amerykańska, prosty przepis, pieczenie, kanapki, przepisy z 5 składników, szybkie danie]",5.0,4.999996,5
2742,"[Dania i przekąski, Pizze]",Łatwe,45 minut,"{'pizza (ciasto + sos)': '1 opak.', 'krewetki black tiger': '10 szt.', 'szynka prosciutto': '10 plastrów', 'pesto': '1 szt.', 'mozarella tarta': '100 g', 'przyprawa w mini kostkach czosnek': '1 szt.'}",Krok 1\nCiasto i sos przygotowujemy zgodnie z zaleceniami na opakowaniu.\nKrok 2\nKiedy ciasto wyrośnie rozciągamy je. Na cieście rozsmarowujemy sos a następnie posypujemy go mozarellą.\nKrok 3\nKażdą krewetkę owijamy połową plasterka szynki i układamy na pizzy. Między krewetkami dajemy po 1/2 łyżeczki pesto. Całość posypujemy posiekaną papryczką i rozkruszoną kosteczką czosnkową.\nKrok 4\nPizzę wstawiamy do piekarnika nagrzanego do 220 stopni i pieczmy ok 15-20 min.,0.0,https://www.przepisy.pl/przepis/pizza-z-krewetkami-pesto-i-prosciutto,"Pizza z krewetkami, pesto i prosciutto",1.0,5.0,"[ciasto na pizzę, mięso, owoce morza, prosty przepis, pieczenie, pizze]",5.0,4.999996,6
2742,"[Dania i przekąski, Pizze]",Łatwe,45 minut,"{'pizza (ciasto + sos)': '1 opak.', 'krewetki black tiger': '10 szt.', 'szynka prosciutto': '10 plastrów', 'pesto': '1 szt.', 'mozarella tarta': '100 g', 'przyprawa w mini kostkach czosnek': '1 szt.'}",Krok 1\nCiasto i sos przygotowujemy zgodnie z zaleceniami na opakowaniu.\nKrok 2\nKiedy ciasto wyrośnie rozciągamy je. Na cieście rozsmarowujemy sos a następnie posypujemy go mozarellą.\nKrok 3\nKażdą krewetkę owijamy połową plasterka szynki i układamy na pizzy. Między krewetkami dajemy po 1/2 łyżeczki pesto. Całość posypujemy posiekaną papryczką i rozkruszoną kosteczką czosnkową.\nKrok 4\nPizzę wstawiamy do piekarnika nagrzanego do 220 stopni i pieczmy ok 15-20 min.,0.0,https://www.przepisy.pl/przepis/pizza-z-krewetkami-pesto-i-prosciutto,"Pizza z krewetkami, pesto i prosciutto",1.0,5.0,"[ciasto na pizzę, mięso, owoce morza, prosty przepis, pieczenie, pizze]",5.0,4.999996,6
2742,"[Dania i przekąski

In [136]:
r_duration = list(r.duration.values)
different_duration = []
for duration in r_duration:
    duration = int(re.sub(r"\D", "", duration))
    if duration not in different_duration:
        different_duration.append(duration)
print(different_duration)

[15, 60, 65, 45, 25, 30, 90, 5, 20, 0, 120, 70, 100, 50, 40, 10, 35, 180, 140, 75, 80, 95, 55, 27, 110, 115, 105, 47, 170, 18, 150, 85, 34, 130, 200, 8, 2, 160, 12, 7, 28, 1, 24, 125, 23, 49, 14, 43, 155, 220, 21, 240, 210, 22, 280]


In [ ]:
max_rating_easiest = easiest["rating3"].max()
best_easiest = easiest[easiest["rating3"] == max_rating_easiest]
best_easiest